In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q transformers scikit-learn datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import json
import re

# 🔢 Preprocesamiento y métricas
from sklearn.preprocessing import LabelEncoder
import evaluate

# 📚 Carga de datasets
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset


# Modelos y entrenamiento con Hugging Face
from transformers import (
    T5ForConditionalGeneration,
    T5TokenizerFast,
    T5Config,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback,
)

# 🔥 PyTorch
import torch
from torch.utils.data import DataLoader

## BERT/RoBERTa

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Cargar el modelo y el tokenizador desde el directorio guardado
model_path = "/content/drive/MyDrive/TFG/models/finetuned_bert-base-uncased"
model_path = "/content/drive/MyDrive/TFG/models/finetuned_roberta-base"
modelo = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Definir frases de prueba
frases_prueba = [
    "Add this song to my playlist",  # ADD
    "Could you remove this track?",  # REMOVE
    "Show me my current playlist",  # VIEW
    "Clear the entire playlist",    # CLEAR
    "Add this song to my playlist",  # ADD
    "Remove this track from my list",  # REMOVE
    "Show me my current playlist",  # VIEW
    "Clear my entire playlist",  # CLEAR
    "Could you please delete all the songs?",  # CLEAR
    "Can you add a song to my playlist?",  # ADD
    "I'd like to see what songs I have added",  # VIEW
    "Take this song out of my list",  # REMOVE
    "Hey, can u clear my playlist",     # CLEAR
    "can u delete the song Crazy",   # REMOVE
]

# Tokenizar las frases de prueba
inputs = tokenizer(frases_prueba, padding=True, truncation=True, return_tensors="pt")

# Pasar las frases al modelo para obtener predicciones
modelo.eval()  # Poner el modelo en modo evaluación
with torch.no_grad():
    outputs = modelo(**inputs)

# Convertir los logits a predicciones
predicciones = torch.argmax(outputs.logits, dim=1).numpy()

# Mapear los índices a las etiquetas
etiquetas = ["add", "clear", "remove", "view"]

# Mostrar las predicciones
for frase, pred in zip(frases_prueba, predicciones):
    print(f"Frase: '{frase}' → Predicción: {etiquetas[pred]}")


Frase: 'Add this song to my playlist' → Predicción: add
Frase: 'Could you remove this track?' → Predicción: remove
Frase: 'Show me my current playlist' → Predicción: view
Frase: 'Clear the entire playlist' → Predicción: clear
Frase: 'Add this song to my playlist' → Predicción: add
Frase: 'Remove this track from my list' → Predicción: remove
Frase: 'Show me my current playlist' → Predicción: view
Frase: 'Clear my entire playlist' → Predicción: clear
Frase: 'Could you please delete all the songs?' → Predicción: clear
Frase: 'Can you add a song to my playlist?' → Predicción: add
Frase: 'I'd like to see what songs I have added' → Predicción: view
Frase: 'Take this song out of my list' → Predicción: remove
Frase: 'Hey, can u clear my playlist' → Predicción: clear
Frase: 'can u delete the song Crazy' → Predicción: remove


## T5

In [ ]:
# Cargar el modelo y el tokenizador
model_path = "/content/drive/MyDrive/TFG/models/finetuned_t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5TokenizerFast.from_pretrained(model_path)

# Lista de frases de prueba
frases_prueba = [
    "Add this song to my playlist",  # ADD
    "Could you remove this track?",  # REMOVE
    "Show me my current playlist",  # VIEW
    "Clear the entire playlist",    # CLEAR
    "Remove this track from my list",  # REMOVE
    "Show me my current playlist",  # VIEW
    "Clear my entire playlist",  # CLEAR
    "Could you please delete all the songs?",  # CLEAR
    "Can you add a song to my playlist?",  # ADD
    "I'd like to see what songs I have added",  # VIEW
    "Take this song out of my list",  # REMOVE
    "Hey, can u clear my playlist",     # CLEAR
    "can u delete the song Crazy",   # REMOVE
]

# Preprocesar y generar predicciones para cada frase
for frase in frases_prueba:
    # Preprocesar la entrada
    input_ids = tokenizer("Clasifica: " + frase, return_tensors="pt").input_ids
    attention_mask = tokenizer("Clasifica: " + frase, return_tensors="pt").attention_mask

    # Generar la predicción
    model.eval()
    with torch.no_grad():
        generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=10)

    # Decodificar la predicción
    decoded_pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # Imprimir la predicción
    print(f"Frase: {frase}")
    print(f"Predicción: {decoded_pred}")
    print("-" * 50)


Frase: Add this song to my playlist
Predicción: add
--------------------------------------------------
Frase: Could you remove this track?
Predicción: remove
--------------------------------------------------
Frase: Show me my current playlist
Predicción: view
--------------------------------------------------
Frase: Clear the entire playlist
Predicción: clear
--------------------------------------------------
Frase: Remove this track from my list
Predicción: clear
--------------------------------------------------
Frase: Show me my current playlist
Predicción: view
--------------------------------------------------
Frase: Clear my entire playlist
Predicción: clear
--------------------------------------------------
Frase: Could you please delete all the songs?
Predicción: clear
--------------------------------------------------
Frase: Can you add a song to my playlist?
Predicción: add
--------------------------------------------------
Frase: I'd like to see what songs I have added
Pred